In [ ]:
# A proposition of implementation of a web crawler. Tought and implemented by Franck & Noela BD4D 2020

Error: Jupyter cannot be started. Error attempting to locate jupyter: 

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import os
from urllib.parse import urlparse
import time
import pandas as pd
from threading import Thread

In [ ]:
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
headers = {'User-Agent': user_agent}

In [ ]:
CONTENT_TYPES = ["application/pdf", "pdf", "word", "excel", "officedocument"]

In [ ]:
links_visited = {} # This keeps the visited links. A dict is faster to look for key existance than a list

In [ ]:
downloaded_files = {"file_name": [], "time_taken": [], "country": []} # dict to be turned into DataFrame

In [ ]:
def get_filename_from_cd(cd, url):
    """
    Get filename from content-disposition
    """
    if not cd:
        # If content disposition not defined, the name of the file is the last part of the url
        if url.find('/'):
            return url.rsplit('/', 1)[1]
    fname = re.findall('filename=(.+)', cd)
    if len(fname) == 0:
        return None
    return fname[0]

In [ ]:
def is_downloadable(response):
    """
    Does the url contain a downloadable resource
    """
    header = response.headers
    content_type = header.get('content-type')
    return content_type.lower() in CONTENT_TYPES

In [ ]:
def write_file_to_disk(country_code, file_name, content):
    file = path = "{}/{}/{}".format(os.getcwd(), country_code, file_name)
    open(file, 'wb').write(content)

In [ ]:
def parse_links(html, base_url, url):
    soup = BeautifulSoup(html, 'lxml')
    parsed_url = urlparse(base_url)
    scheme = parsed_url.scheme
    root_url = parsed_url.netloc
    to_visit = []
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        new_link = None
        if href:
            if href[0] == "/" :
                # example of href = "/environment.html
                new_link = scheme + '://' + root_url + href
            elif (root_url in href):
                # example of href = "**root_url**/href"
                new_link = href
            elif (not re.match('^https?', href)) and re.match('[a-zA-Z]', href):
                # example of href "home.html" should exclude link starting with http which may link outside (youtube, facebook...)
                next_link = url + "/" + href
                if root_url in next_link:
                    new_link = next_link
            if new_link and links_visited.get(new_link) == None:
                # this is to avoid visiting the same links repeatedly
                to_visit.append(new_link)

    return to_visit

In [ ]:
def visit_link(url=None, country_code="", base_url=""):
    if links_visited.get(url) != None:
        return
    else:
        print('Visiting: ' + url)
        start = time.clock()
        links_visited[url] = 1
        try:
            response = requests.get(url, allow_redirects=True, headers=headers)
            if response.status_code == 200:
                if is_downloadable(response):
                    file_name = response.headers.get('content-disposition')
                    file_name = get_filename_from_cd(file_name, url)
                    print(file_name + ' YEAH')
                    write_file_to_disk(country_code, file_name, response.content)
                    downloaded_files['country'].append(country_code)
                    downloaded_files['file_name'].append(file_name)
                    downloaded_files['time_taken'].append((time.clock() - start) * 1000)
                    df = pd.DataFrame(downloaded_files)
                    # We write to the file each time in other to have results in case of early exit...
                    df.to_csv('result.csv', encoding='utf-8', index=False)
                else:
                    html = response.text
                    links_to_visit = parse_links(html, base_url, url)
                    for link in links_to_visit:
                        visit_link(link, country_code, base_url)
                    return 0
        except:
            print("Link not valid")

In [ ]:
def create_country_folder(folder_name):
    current_path = os.getcwd()
    path = "{}/{}".format(current_path, folder_name)
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
threads = []
def process_countries():
    df = pd.read_csv('../data/webLinks.csv')
    for index, row in df.iterrows():
        # create the folder for the country
        create_country_folder(row['countryCode'])
        if row['linkAvailable'] == 'Yes':
            # I make use of Threads for parallel execution (instead of one country as a loop will normally do)
            process = Thread(target=visit_link, args=[row['link'], row['countryCode'], row['link']])
            process.start()
            threads.append(process)
        else:
            pass

# We now pause execution on the main thread by 'joining' all of our started threads.
# This ensures that each has finished processing the urls.
for process in threads:
    process.join()


In [ ]:
process_countries()